<a href="https://colab.research.google.com/github/dastanrab/Data-Structures/blob/master/calori_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth
!pip install bitsandbytes
!pip install trl
!pip install accelerate
!pip install datasets
!pip install transformers
!pip install protobuf==3.20.3


from unsloth import FastLanguageModel
from datasets import load_dataset

# Load base model with Unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Phi-3-mini-4k-instruct-bnb-4bit',
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True
)

# Load dataset directly from Hugging Face
dataset = load_dataset("Codatta/MM-Food-100K", split="train")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.9.2: Fast Mistral patching. Transformers: 4.56.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
import json
# Map dataset to text format for SFTTrainer
def to_text(ex):
    # ورودی (پرومپت) از ستون‌های دیتاست ساخته میشه
    prompt = (
        f"Dish: {ex['dish_name']}\n"
        f"Ingredients: {', '.join(ex['ingredients'])}\n"
        f"Portion: {', '.join(ex['portion_size'])}\n"
        f"Cooking method: {ex['cooking_method']}"
    )

    # خروجی (ریسپانس) پروفایل غذاییه
    response = json.dumps(ex["nutritional_profile"], ensure_ascii=False)

    msgs = [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response},
    ]
    return {
        "text": tokenizer.apply_chat_template(
            msgs, tokenize=False, add_generation_prompt=False
        )
    }

dataset = dataset.map(to_text, remove_columns=dataset.column_names)

In [3]:
# Prepare model for LoRA fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = 'none',
    use_gradient_checkpointing = 'unsloth'
)

Unsloth 2025.9.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    dataset_text_field = 'text',
    max_seq_length = 2048,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,  # small for demo, increase for real training
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        num_train_epochs = 1
    ),
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)
wandb: Currently logged in as: dastanrab (dastanrab-bazist) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.692000
2,1.699500
3,1.505100
4,1.603200
5,1.372100
6,1.372500
7,1.292600
8,1.177400
9,1.089500
10,0.994800


TrainOutput(global_step=60, training_loss=0.5505928811927636, metrics={'train_runtime': 305.2555, 'train_samples_per_second': 1.572, 'train_steps_per_second': 0.197, 'total_flos': 3287130548207616.0, 'train_loss': 0.5505928811927636, 'epoch': 0.0048})

In [5]:
# Test inference
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "Dish: Fried Chicken\nIngredients: chicken, breading, oil\nPortion: 300g\nCooking method: Frying"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
    do_sample=True
)

response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Dish: Fried Chicken
Ingredients: chicken, breading, oil
Portion: 300g
Cooking method: Frying "{\"fat_g\":20.0,\"protein_g\":30.0,\"calories_kcal\":500,\"carbohydrate_g\":30.0}"


In [ ]:
# Export to GGUF for Ollama
model.save_pretrained_gguf(
    "gguf_food_model",
    tokenizer,
    quantization_method="q4_k_m",
    maximum_memory_usage = 0.3
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 1.08 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [01:52<00:00,  3.52s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving gguf_food_model/pytorch_model.bin...
Done.
